In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Nombres de los archivos parquet
archivos_parquet = ['clean_df_ur.parquet', 'definitivo_df_ui.parquet', 'definitivo_df_sg.parquet']

# Cargar el primer archivo parquet
df_ur = pd.read_parquet('DataSet/' + archivos_parquet[0])

# Cargar el segundo archivo parquet
df_ui = pd.read_parquet('DataSet/' + archivos_parquet[1])

# Cargar el tercer archivo parquet
df_sg = pd.read_parquet('DataSet/' + archivos_parquet[2])

In [3]:
# Combinar df_ui y df_sg
combined_df = pd.merge(df_ui[['item_id', 'playtime_forever_log']], 
                       df_sg[['id', 'price']], 
                       left_on='item_id', 
                       right_on='id')

# Seleccionar las columnas que representarán las características
game_features = combined_df[['playtime_forever_log', 'price']]

# verificar características
print(game_features.head())


   playtime_forever_log  price
0              1.945910   9.99
1              4.543295   9.99
2              4.691348   9.99
3              5.796058   9.99
4              8.744488   9.99


In [4]:
# Aplicar PCA para reducir la dimensionalidad
pca = PCA(n_components=2)  
game_features_reduced = pca.fit_transform(game_features)


In [5]:
def recomendacion_juego(game_id, game_features_reduced, df_ui, top_n=5):
    # Encuentra el vector de características para el juego dado
    game_vector = game_features_reduced[game_id:game_id+1]

    # Calcula la similitud del coseno entre este juego y todos los demás
    cosine_similarities = cosine_similarity(game_vector, game_features_reduced)

    # Obtener los índices de los juegos más similares
    similar_games_indices = cosine_similarities.argsort()[0][-top_n-1:-1][::-1]

    # Excluir el propio juego de las recomendaciones
    similar_games_indices = [i for i in similar_games_indices if i != game_id]

    # Mapear los índices a nombres de juegos
    recommended_games = df_ui.loc[similar_games_indices, 'item_name']

    return recommended_games[:top_n].tolist()




In [6]:
# Ejemplo de uso
id_del_juego = 227300  
print(recomendacion_juego(id_del_juego, game_features_reduced, df_ui))

['Kerbal Space Program', "Tom Clancy's Ghost Recon Phantoms - NA", 'Guncraft', 'Tropico 5', 'VVVVVV']


In [ ]:
# Combinar df_ui y df_sg
combined_df = pd.merge(df_ui[['item_id', 'playtime_forever_log']], 
                       df_sg[['id', 'price']], 
                       left_on='item_id', 
                       right_on='id')

# Seleccionar las columnas que representarán tus características
game_features = combined_df[['playtime_forever_log', 'price']]

# Aplicar PCA para reducir la dimensionalidad
pca = PCA(n_components=2)  # Ajusta el número de componentes según sea necesario
game_features_reduced = pca.fit_transform(game_features)

def recomendacion_juego(game_id, game_features_reduced, df_ui, top_n=5):
    # Encuentra el vector de características para el juego dado
    game_vector = game_features_reduced[game_id:game_id+1]

    # Calcula la similitud del coseno entre este juego y todos los demás
    cosine_similarities = cosine_similarity(game_vector, game_features_reduced)

    # Obtener los índices de los juegos más similares
    similar_games_indices = cosine_similarities.argsort()[0][-top_n-1:-1][::-1]

    # Excluir el propio juego de las recomendaciones
    similar_games_indices = [i for i in similar_games_indices if i != game_id]

    # Mapear los índices a nombres de juegos
    recommended_games = df_ui.loc[similar_games_indices, 'item_name']

    return recommended_games[:top_n].tolist()

# Ejemplo de uso
id_del_juego = 227300  
print(recomendacion_juego(id_del_juego, game_features_reduced, df_ui))